In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
import fastbook

In [3]:
from fastbook import *
from fastai.vision.widgets import *

In [4]:
path = Path('/kaggle/input/skin-cancer-malignant-vs-benign/train')

In [5]:
path

In [6]:
class DataLoaders(GetAttr):
    def __init__(self, *loaders): self.loaders = loaders
    def __getitem__(self, i): return self.loaders[i]
    train,valid = add_props(lambda i,self: self[i])

In [7]:
imges = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [8]:
dls = imges.dataloaders(path)

In [9]:
dls.valid.show_batch(max_n=4, nrows=1)

In [10]:
imges = imges.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = imges.dataloaders(path)

In [11]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(15)

In [12]:
learn.fine_tune(15)

In [13]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [14]:
interp.plot_top_losses(5, nrows=1)

In [15]:
learn.export()

In [16]:
path = Path()
path.ls(file_exts='.pkl')

In [19]:
learn_inf = load_learner(path/'export.pkl')

In [21]:
learn_inf.predict('/kaggle/input/skin-cancer-malignant-vs-benign/test/benign/211.jpg')

In [22]:
btn_upload = widgets.FileUpload()
btn_upload

In [23]:
img = PILImage.create(btn_upload.data[-1])

In [24]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [25]:
pred,pred_idx,probs = learn_inf.predict(img)

In [26]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [27]:
btn_run = widgets.Button(description='Classify')
btn_run

In [28]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [29]:
btn_upload = widgets.FileUpload()

In [30]:
VBox([widgets.Label('Select your image!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [1]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila